# Quick Examples for OntoGPT

Last updated Jul 19, 2024

The following examples demonstrate basic functionality of OntoGPT and the SPIRES method. They may be run through the command line or the webapp.

## Setup

In [ ]:
%pip install ontogpt[web]

Set your OpenAI API key:

In [ ]:
openai_api_key = "<your openai api key here>"
!runoak set-apikey -e openai {openai_api_key}

To start the web interface, run the following:

In [ ]:
# If this is running correctly, you'll see several INFO messages,
# including one that says 'Uvicorn running on http://127.0.0.1:8000'.
# Click the link to open the web interface in your browser.
# Stop the server by interrupting the notebook kernel (press the stop button adjacent to this cell).
!web-ontogpt

## Example 1: Extracting FOODON terms

The following paragraph contains names of food items we will extract with OntoGPT.

In [ ]:
text = "Spain is a significant contributor to the global agricultural sector, with its primary exports demonstrating the country's diverse climate and agricultural capacity. The nation leads the world in olive oil production and export, renowned for its quality. Spain is also one of the top wine producers globally, with notable regions including Rioja and Ribera del Duero. It exports a wide range of fruits, particularly citrus fruits such as oranges and lemons from Valencia, as well as peaches, strawberries, melons, and apples. Vegetable exports are substantial, with tomatoes, peppers, cucumbers, and lettuce being predominant, especially from the Almería region known for its greenhouse production. Spain is recognized for its premium pork products, notably jamón ibérico and jamón serrano, and is a leading exporter of nuts, particularly almonds and hazelnuts. Additionally, Spain exports significant quantities of fish and seafood, including sardines, tuna, and shrimp, facilitated by its extensive coastline and fishing traditions."
with open('text.txt', 'w') as outfile:
    outfile.write(text)

We will attempt to ground each extracted term to FOODON or CHEBI.
Conveniently, OntoGTP includes a template (`foodon_simple`) for this task.

### CLI

In [ ]:
import yaml

In [ ]:
!ontogpt -v extract -t foodon_simple -i text.txt -m gpt-4o -o output1.yaml

In [ ]:
# Read the output file
with open('output1.yaml', 'r') as infile:
    output1 = yaml.safe_load(infile)
for entity in output1["named_entities"]:
    if ((entity["id"]).split(":"))[0] not in ["CHEBI", "FOODON"]:
        print("NOT GROUNDED -> " + entity["label"])
    else:
        print(entity["id"], entity["label"])

### Webapp

Run the webapp as above, then select `foodon_simple` from the list of schemas, `gpt-4o` from the list of LLMs, and copy+paste the following into the text box.

_Spain is a significant contributor to the global agricultural sector, with its primary exports demonstrating the country's diverse climate and agricultural capacity. The nation leads the world in olive oil production and export, renowned for its quality. Spain is also one of the top wine producers globally, with notable regions including Rioja and Ribera del Duero. It exports a wide range of fruits, particularly citrus fruits such as oranges and lemons from Valencia, as well as peaches, strawberries, melons, and apples. Vegetable exports are substantial, with tomatoes, peppers, cucumbers, and lettuce being predominant, especially from the Almería region known for its greenhouse production. Spain is recognized for its premium pork products, notably jamón ibérico and jamón serrano, and is a leading exporter of nuts, particularly almonds and hazelnuts. Additionally, Spain exports significant quantities of fish and seafood, including sardines, tuna, and shrimp, facilitated by its extensive coastline and fishing traditions._

Click Submit when ready. OntoGPT will require a moment to retrieve the FOODON ontology and retrieve results.

## Example 2: Extracting Environmental Sample Data

This time, we'll use a more complex template, and we'll use it to extract data about samples collected in a study of heavy metal contamination of food crops and beyond by Muimba-Kankolongo et al (https://pubmed.ncbi.nlm.nih.gov/35844936/).

The template we will use incorporates annotator ontologies from Bioportal (https://bioportal.bioontology.org/), so you will need to specify a Bioportal API key as follows:

In [ ]:
bioportal_api_key = "<your bioportal api key here>"
!runoak set-apikey -e bioportal {bioportal_api_key}

Alternatively, wherever the following examples specify the `environmental_sample` template, substitute the `environmental_sample_ungrounded` template name instead.
As the name implies, this template will only extract terms and relations but will not attempt to ground them.

### CLI

For convenience, let's save a local copy of the paper's abstract.

In [ ]:
abs = "Zambia and the DR Congo are situated in the central African Copperbelt, which is part of the Lufilian geological structure arc stretching over from Kolwezi in Katanga Province in the DRC to Luanshya in Copperbelt Province in Zambia. The area has large copper-cobalt deposits of which the extraction causes severe ecosystem damage due to pollution of water, food crops, and the ambient air negatively impacting population health. Contamination of drinking water for domestic use and foods (cereals, roots and tubers, vegetables, and fruits) was determined by assessing the contents of trace metals including Mn, Ni, Pb, Zn, Co, As, U, Cd, and Cu and through a questionnaire for environmental damage. Food samples were analyzed by inductively coupled argon plasma/optical emission spectroscopy (ICP-OES), while water and urine samples were analyzed by inductively coupled argon plasma mass spectrometry (ICP-MS). Concentrations of Ni, Pb, and Cd were higher in almost all food crops, although Cu was more in samples of Cucurbita maxima and Amaranthus hybridus. Mean contents (μg/L) of Mn, Zn, Cd, Pb, and U were, respectively, 5,454.6, 2552.2, 138.7, 39.7, 2361.1, and 21.4 in the DRC and 108.9, 543.3, 0.3, 0.2, 1.5, and 0.5 in Zambia, being significantly higher and always far above World Health Organization maximum limits in the DRC. Urine samples taken only from the DRC contained trace metals with children's samples being more contaminated than adult ones. Our results conclusively echo the most critical challenges of toxic pollutants causing numerous health issues among the population. Given an outcry among households adjacent to mines about land degradation and food spoilage, and health problems over years, joint efforts are needed from public and private sectors for stringent mining exploitation monitoring for sustainable governance to protect the environment and ensure food and nutrition safety, and population well-being in Zambia and the DR Congo."
with open('text2.txt', 'w') as outfile:
    outfile.write(abs)

In [ ]:
# This may take some time to complete.
!ontogpt -v extract -t environmental_sample -i text2.txt -m gpt-4o -o output2.yaml

This paper has a PMC full text available, so we may also use that as input, in the hopes of getting a more comprehensive result.

In [ ]:
!ontogpt -v pubmed-annotate -t environmental_sample -m gpt-4o -o output3.yaml --get-pmc --limit 2 "35844936"

The output will be saved in the file `output3.yaml`. Note that each "chunk" of the input text is processed separately, so the output includes one set of extractions for each. This makes it easier to identify which entity or relation corresponds with a given text section, though it also means there will likely be redundancy.

### Webapp

Run the webapp as above (web-ontogpt), then select `environmental_sample` from the list of schemas, `gpt-4o` from the list of LLMs, and copy+paste the following abstract (from Muimba-Kankolongo et al (2022) J Environ Public Health; PMID:35844936) into the text box.

_Zambia and the DR Congo are situated in the central African Copperbelt, which is part of the Lufilian geological structure arc stretching over from Kolwezi in Katanga Province in the DRC to Luanshya in Copperbelt Province in Zambia. The area has large copper-cobalt deposits of which the extraction causes severe ecosystem damage due to pollution of water, food crops, and the ambient air negatively impacting population health. Contamination of drinking water for domestic use and foods (cereals, roots and tubers, vegetables, and fruits) was determined by assessing the contents of trace metals including Mn, Ni, Pb, Zn, Co, As, U, Cd, and Cu and through a questionnaire for environmental damage. Food samples were analyzed by inductively coupled argon plasma/optical emission spectroscopy (ICP-OES), while water and urine samples were analyzed by inductively coupled argon plasma mass spectrometry (ICP-MS). Concentrations of Ni, Pb, and Cd were higher in almost all food crops, although Cu was more in samples of Cucurbita maxima and Amaranthus hybridus. Mean contents (μg/L) of Mn, Zn, Cd, Pb, and U were, respectively, 5,454.6, 2552.2, 138.7, 39.7, 2361.1, and 21.4 in the DRC and 108.9, 543.3, 0.3, 0.2, 1.5, and 0.5 in Zambia, being significantly higher and always far above World Health Organization maximum limits in the DRC. Urine samples taken only from the DRC contained trace metals with children's samples being more contaminated than adult ones. Our results conclusively echo the most critical challenges of toxic pollutants causing numerous health issues among the population. Given an outcry among households adjacent to mines about land degradation and food spoilage, and health problems over years, joint efforts are needed from public and private sectors for stringent mining exploitation monitoring for sustainable governance to protect the environment and ensure food and nutrition safety, and population well-being in Zambia and the DR Congo._

Click Submit when ready. OntoGPT will require some time to retrieve the necessary ontologies and retrieve results.